# Gulf Marine Mammals

This indicator was used in the Southeast Blueprint 2023.
Code written by Rua Mordecai and Amy Keister

In [1]:
import os
import arcpy

In [2]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
OutWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\ContinentalSoutheastBlueprint2023_FinalIndicators\ContinentalSoutheastBlueprint2023_FinalIndicators\SpatialData"

In [3]:
# define final output name
IndicatorFileName = "GulfMarineMammals.tif"

In [4]:
# define raster used for cell size, extent, and snapping
MarineRaster = r"D:\Blueprint\2023\extent\Marine_Extent_v1.tif"
MarineSubregion = r"D:\Blueprint\2023\extent\Marine_GoM_v1.tif"
MarineSubregionVector = r"D:\Blueprint\2023\extent\Marine_GoM_v1.shp"

In [5]:
# define inputs
mammal = r"D:\Blueprint\2023\Indicators\GulfMarineMammals\zonation\rankmap.tif"
nlcd = r"D:\Landcover\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"

In [6]:
# species data folder
speciesFolder = "D:/Blueprint/2023/Indicators/GulfMarineMammals/speciesData"

In [7]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

### Start analysis

### Prep data for Zonation

In [11]:
# Go to folder with species data
arcpy.env.workspace = speciesFolder

In [12]:
# find all the species shapefiles
speciesFileList = arcpy.ListFeatureClasses()
print(speciesFileList)

['Beaked_Whales_Monthly_Density.shp', 'Blackfish_Monthly_Density_wUnids.shp', 'Clymene_Dolphin_Monthly_Density_wUnids.shp', 'Oceanic_AtlanticSpotted_Dolphin_Monthly_Density_wUnids.shp', 'Oceanic_CommonBottlenose_Dolphin_Monthly_Density_wUnids.shp', 'PantropicalSpotted_Dolphin_Monthly_Density_wUnids.shp', 'Pilot_Whales_Monthly_Density_wUnids.shp', 'Pygmy_Dwarf_SpermWhales_Monthly_Density.shp', 'Rices_Whale_Monthly_Density.shp', 'Rissos_Dolphin_Monthly_Density_wUnids.shp', 'Shelf_AtlanticSpotted_Dolphin_Monthly_Density_Unclipped.shp', 'Shelf_CommonBottlenose_Dolphin_Monthly_Density_Unclipped.shp', 'Sperm_Whale_Monthly_Density.shp', 'Spinner_Dolphin_Monthly_Density_wUnids.shp', 'Striped_Dolphin_Monthly_Density_wUnids.shp']


In [13]:
# make another list with the full directory path to species files
SpeciesFilesWithDirectory = []
for i in range(len(speciesFileList)):
    SpeciesFilesWithDirectory.append(speciesFolder + "/" + speciesFileList[i])

In [8]:
# Go to the output workspace
arcpy.env.workspace = OutWorkspace

In [36]:
# make copies of the source data
for i in range(len(speciesFileList)):
    arcpy.CopyFeatures_management(SpeciesFilesWithDirectory[i], speciesFileList[i])

In [37]:
# Replace null value -9999 with 0 based on ESRI script 
for i in range(len(speciesFileList)):
    path = speciesFileList[i]
    fieldObs = arcpy.ListFields(path)  
    fieldNames = []  
    for field in fieldObs:  
        fieldNames.append(field.name)  
    del fieldObs  
    fieldCount = len(fieldNames)  

    with arcpy.da.UpdateCursor(path, fieldNames) as curU:  
        for row in curU:  
            rowU = row  
            for field in range(fieldCount):  
                if rowU[field] == -9999:  
                    rowU[field] = 0           
            curU.updateRow(rowU)
    del curU

In [9]:
# Shapefile fields for monthly data
months = ["Jan_n","Feb_n","Mar_n","Apr_n","May_n","Jun_n","Jul_n","Aug_n","Sep_n","Oct_n","Nov_n","Dec_n"]
pygmySpermWhaleMonths = ["Apr_n","May_n","Jun_n","Jul_n","Aug_n","Sep_n","Oct_n","Nov_n"]

In [40]:
# Loop through all the species and months and make rasters. This takes about 12 hours to finish and makes some big files
for i in range(len(speciesFileList)):
    if speciesFileList[i] == 'Pygmy_Dwarf_SpermWhales_Monthly_Density.shp':
        # Convert monthly species info for this species, which includes less months, to rasters
        for mon in pygmySpermWhaleMonths:
            outRasterName=speciesFileList[i][:-4]+mon+".tif"
            arcpy.conversion.FeatureToRaster(in_features=speciesFileList[i], field=mon, out_raster=outRasterName, cell_size=MarineSubregion)
    else:
        # Convert monthly species info to rasters
        for mon in months:
            outRasterName=speciesFileList[i][:-4]+mon+".tif"
            arcpy.conversion.FeatureToRaster(in_features=speciesFileList[i], field=mon, out_raster=outRasterName, cell_size=MarineSubregion)       
        

### Make Zonation mask

In [9]:
# Reclass to make mask for analysis area
with arcpy.EnvManager(outputCoordinateSystem="Beaked_Whales_Monthly_DensityJan_n.tif", extent="Beaked_Whales_Monthly_DensityJan_n.tif", snapRaster="Beaked_Whales_Monthly_DensityJan_n.tif", cellSize="Beaked_Whales_Monthly_DensityJan_n.tif"):
    out_raster = arcpy.sa.Reclassify(nlcd, "Value", "0 11 1;12 95 NODATA"); out_raster.save("nlcdWaterMask.tif")

In [10]:
# Dissolve sample polygons to get sampled area
arcpy.management.Dissolve("Beaked_Whales_Monthly_Density.shp", "MammalSampleArea.shp")

<Result 'D:\\Blueprint\\2023\\Indicators\\GulfMarineMammals\\working\\MammalSampleArea.shp'>

In [11]:
# extract only sampled area that's part of NLCD water mask. This takes out shore pixels and a bunch of the pixels outside
# of the US. Match the dimensions of the turtle layers going into Zonation
with arcpy.EnvManager(outputCoordinateSystem="Beaked_Whales_Monthly_DensityJan_n.tif", extent="Beaked_Whales_Monthly_DensityJan_n.tif", snapRaster="Beaked_Whales_Monthly_DensityJan_n.tif", cellSize="Beaked_Whales_Monthly_DensityJan_n.tif"):
    out_raster = arcpy.sa.ExtractByMask("nlcdWaterMask.tif", "MammalSampleArea.shp"); out_raster.save("mammalZonationMask.tif")

In [12]:
# not sure why the last step doesn't result in the right dimensions. This step, without using environmental settings works.
out_raster = arcpy.sa.ExtractByMask("mammalZonationMask.tif", "Beaked_Whales_Monthly_DensityJan_n.tif"); out_raster.save("mammalZonationMask2.tif")

In [10]:
# Convert sampled area to a raster. This is used later to mark small areas of land in the 
# sampling extent as 0
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    arcpy.conversion.FeatureToRaster(in_features="MammalSampleArea.shp", field="Id", out_raster="zero.tif", cell_size=MarineRaster)

In [9]:
# Reclass to make mask of only land to use in zero/noData
with arcpy.EnvManager(outputCoordinateSystem="Beaked_Whales_Monthly_DensityJan_n.tif", extent="Beaked_Whales_Monthly_DensityJan_n.tif", snapRaster="Beaked_Whales_Monthly_DensityJan_n.tif", cellSize="Beaked_Whales_Monthly_DensityJan_n.tif"):
    out_raster = arcpy.sa.Reclassify(nlcd, "Value", "0 11 NODATA;12 95 0"); out_raster.save("nlcdLandMask.tif")

In [10]:
# make raster of zeros to include land removed but also match southern extent of nlcd mask used in Zonation
out_raster = arcpy.sa.Times("nlcdLandMask.tif", "zero.tif"); out_raster.save("zeros.tif")

### Continue prepping data

In [15]:
# I originally thought the big floating point rasters were too big for Zonation to handle. 
# That wasn't the case but I still ended up using the outputs from this in the next step for
# shelfAtlanticSpottedDolphin and shelfBottlenoseDolphin. In theory, it should be possible to skip this step
# and just do the next step of fixing the dimensions for those two species. It's just take a couple name changes.
# Loop through species and months and make smaller rasters.
for i in range(len(speciesFileList)):
    if speciesFileList[i] == 'Pygmy_Dwarf_SpermWhales_Monthly_Density.shp':
        # Convert monthly species info for this species, which includes less months, to smaller rasters
        for mon in pygmySpermWhaleMonths:
            outRasterName="fixed"+speciesFileList[i][:-4]+mon+".tif"
            inRasterName=speciesFileList[i][:-4]+mon+".tif"
            arcpy.management.CopyRaster(inRasterName, outRasterName, '', None, "", "NONE", "NONE", "16_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")            
    else:
        # Convert monthly species info to smaller rasters
        for mon in months:
            outRasterName="fixed"+speciesFileList[i][:-4]+mon+".tif"
            inRasterName=speciesFileList[i][:-4]+mon+".tif"
            arcpy.management.CopyRaster(inRasterName, outRasterName, '', None, "", "NONE", "NONE", "16_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")                   

In [16]:
# Fix size issues with spotted and bottlenose dolphins on the shelf
for mon in months:
    shelfSpottedDolphinFile = "fixedShelf_AtlanticSpotted_Dolphin_Monthly_Density_Unclipped"+mon+".tif"
    fixName = "s"+shelfSpottedDolphinFile
    with arcpy.EnvManager(snapRaster="Beaked_Whales_Monthly_DensityJan_n.tif"):
        out_raster = arcpy.sa.ExtractByMask(shelfSpottedDolphinFile, "Beaked_Whales_Monthly_DensityJan_n.tif"); out_raster.save(fixName)
    shelfBottlenoseDolphinFile = "fixedShelf_CommonBottlenose_Dolphin_Monthly_Density_Unclipped"+mon+".tif"
    fixName = "s"+shelfBottlenoseDolphinFile
    with arcpy.EnvManager(snapRaster="Beaked_Whales_Monthly_DensityJan_n.tif"):
        out_raster = arcpy.sa.ExtractByMask(shelfBottlenoseDolphinFile, "Beaked_Whales_Monthly_DensityJan_n.tif"); out_raster.save(fixName) 

### Bring in zonation results

In [9]:
# take zonation results times 100
out_raster = arcpy.sa.Times(mammal, 100); out_raster.save("mammal.tif")

In [10]:
# convert to integer
out_raster = arcpy.sa.Int("mammal.tif"); out_raster.save("indicator1.tif")

In [9]:
# Reclassify into bins and align with Marine extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    out_raster = arcpy.sa.Reclassify('indicator1.tif', "Value", "0 10 1;11 20 2;21 30 3;31 40 4;41 50 5;51 60 6;61 70 7;71 80 8;81 90 9;91 100 10", "DATA"); out_raster.save("indicator2.tif")

In [11]:
# add back in small areas removed from sampling extent b/c they were on land. These are 0.
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    out_raster = arcpy.sa.CellStatistics("zeros.tif;indicator2.tif", "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save(IndicatorFileName)

### Finalize indicator



In [9]:
# set code block for next step
codeblock = """
def Reclass(v):
    if v == 10:
        return '10 = >90th percentile of importance for marine mammal index species (across larger analysis area)'
    elif v == 9:
        return '9 = >80th-90th percentile of importance'  
    elif v == 8:
        return '8 = >70th-80th percentile of importance'  
    elif v == 7:
        return '7 = >60th-70th percentile of importance'
    elif v == 6:
        return '6 = >50th-60th percentile of importance'
    elif v == 5:
        return '5 = >40th-50th percentile of importance'
    elif v == 4:
        return '4 = >30th-40th percentile of importance'  
    elif v == 3:
        return '3 = >20th-30th percentile of importance'  
    elif v == 2:
        return '2 = >10th-20th percentile of importance'
    elif v == 1:
        return '1 = ≤10th percentile of importance'
    elif v == 0:
        return '0 = Land'
        
"""

In [10]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(IndicatorFileName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\ContinentalSoutheastBlueprint2023_FinalIndicators\\ContinentalSoutheastBlueprint2023_FinalIndicators\\SpatialData\\GulfMarineMammals.tif'>

In [14]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 10:
		return 217
	if Value == 9:
		return 225
	if Value == 8:
		return 233
	if Value == 7:
		return 240
	if Value == 6:
		return 244
	if Value == 5:
		return 247
	if Value == 4:
		return 250
	if Value == 3:
		return 251
	if Value == 2:
		return 253
	if Value == 1:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 10:
		return 52
	if Value == 9:
		return 73
	if Value == 8:
		return 102
	if Value == 7:
		return 131
	if Value == 6:
		return 157
	if Value == 5:
		return 181
	if Value == 4:
		return 204
	if Value == 3:
		return 227
	if Value == 2:
		return 247
	if Value == 1:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 10:
		return 139
	if Value == 9:
		return 129
	if Value == 8:
		return 119
	if Value == 7:
		return 113
	if Value == 6:
		return 113
	if Value == 5:
		return 120
	if Value == 4:
		return 136
	if Value == 3:
		return 158
	if Value == 2:
		return 185
	if Value == 1:
		return 217
	else:
		return 255
		
"""

In [15]:
# calculate Red field
arcpy.management.CalculateField(IndicatorFileName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(IndicatorFileName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(IndicatorFileName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\Blueprint\\2023\\Indicators\\GulfMarineMammals\\working\\GulfMarineMammals.tif'>